In [1]:
import sys

from time import time
from dotenv import load_dotenv

sys.path.append("/Users/alexeylukyanov/Projects/WeatherData/src")
from weathergov.constants import APP_NAME
from weathergov.utils.redis_utils import RedisClient, RedisKeys

In [2]:
load_dotenv()

True

In [3]:
rc = RedisClient()

In [4]:
rc.rc.ping()

True

Our goal is to create a script that does two commands as a transaction

First, create a test sorted set

In [140]:
tsi1 = int(time() * 1000)

In [141]:
tsi1

1721075844983

In [142]:
tsi2 = int(time() * 1000)

In [143]:
tsi2

1721075846303

In [144]:
rc.rc.zadd("TestSet", {"val3": tsi1, "val4": tsi2})

2

In [145]:
rc.rc.zscore("TestSet", "val10")

In [157]:
rc.rc.zrange("TestSet", 0, -1, withscores=True)

[('val2', 1721076646117.854),
 ('val3', 1721076648107.386),
 ('val4', 1721076649350.741)]

In [31]:
script_text = f"""
return "Holla amigo"
"""

In [35]:
script = rc.rc.register_script(script_text)

In [36]:
script()

'Holla amigo'

In [150]:
rc.rc.eval("""
-- Get the station ID with the latest update time
local res = redis.call("ZPOPMIN", "TestSet")
local time = redis.call("TIME")

-- Set the key to the current timestamp
redis.call("ZADD", "TestSet", time[1] * 1000 + time[2] / 1000, res[1])

-- Return the last time when it calculations were done
return res
""", 0)

['val3', '1721075844983']

In [148]:
[('val2', 1721075793558.0),
 ('val3', 1721075844983.0),
 ('val4', 1721075846303.0)]

[('val2', 1721075793558.0),
 ('val3', 1721075844983.0),
 ('val4', 1721075846303.0)]

In [152]:
script = rc.rc.register_script("""
        -- Get the station ID with the latest update time
        local res = redis.call("ZPOPMIN", "TestSet")
        local time = redis.call("TIME")

        -- Set the key to the current timestamp
        redis.call("ZADD", "TestSet", time[1] * 1000 + time[2] / 1000, res[1])

        -- Return the last time when it calculations were done
        return res
        """)

In [156]:
script()

['val4', '1721076641415.421']

### Check how many keys we actually have right now, before the stations load

In [6]:
rc.script1()

['0007W', '-1']

In [7]:
rc.rc.zscore(RedisKeys.STATION_RT_TABLE, "0007W")

1721080725618.104

In [8]:
rc.rc.zrange("TestSet", 0, -1, withscores=True)

[('val2', 1721076646117.854),
 ('val3', 1721076648107.386),
 ('val4', 1721076649350.741)]

In [14]:
type(rc.rc.zscore("TestSet", "val2"))

float

In [11]:
res = rc.rc.zscore("TestSet", "val20")

In [12]:
res is None

True

In [15]:
rc.rc.get("weather_station:weather.gov:last_data_update_ts")

'1720890011'

In [16]:
rc.rc.set("weather_station:weather.gov:last_data_update_ts", 0)

True

In [18]:
len(rc.rc.zrange(RedisKeys.STATION_RT_TABLE, 0, -1))

46483

In [19]:
rc.rc.llen("weather_station_process_queue")

1113